In [5]:
from sqlite3 import dbapi2 as sq3
from pathlib import Path
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(Path(PATHSTART) / dbfile)
    return sqlite_db

In [6]:
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

In [7]:
from collections import OrderedDict
import pandas as pd
def make_frame(list_of_tuples, legend):
    framelist=[]
    for i, cname in enumerate(legend):
        framelist.append((cname,[e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_dict(OrderedDict(framelist))

In [10]:
def make_query(sel):
    c=db.cursor().execute(sel)
    return c.fetchall()

In [14]:
db=init_db("cancont.db", "")

In [16]:
candidate_cols = [e[1] for e in make_query("PRAGMA table_info(candidates);")]
candidate_cols

['id', 'first_name', 'last_name', 'middle_name', 'party']

In [17]:
out=make_query("SELECT first_name, party FROM candidates;")
make_frame(out,['first_name', 'amount'])

,first_name,amount
0,Mike,R
1,Barack,D
2,Rudolph,R
3,Mike,D
4,John,D
5,Bill,D
6,Duncan,R
7,Dennis,D
8,Ron,R
9,Joseph,D


In [37]:
contributor_cols = [e[1] for e in make_query("PRAGMA table_info(contributors);")]
contributor_cols

['id',
 'last_name',
 'first_name',
 'middle_name',
 'street_1',
 'street_2',
 'city',
 'state',
 'zip',
 'amount',
 'date',
 'candidate_id']

In [38]:
rem="""
DELETE FROM contributors;
"""
c=db.cursor().execute(rem)
db.commit()

In [39]:
ins="""
INSERT INTO contributors (last_name, first_name, middle_name, street_1, street_2, city, state, zip, amount, date, candidate_id) \
    VALUES (?,?,?,?,?, ?, ?, ?, ?, ?, ?);
"""

In [40]:
slines[1]

['',
 'Agee',
 'Steven',
 '',
 '549 Laurel Branch Road',
 '',
 'Floyd',
 'VA',
 '24091',
 '500.00',
 '2007-06-30',
 '16']

In [41]:
from datetime import date

fd = open("data/contributors_with_candidate_id.txt")
slines =[l.strip().split('|') for l in fd.readlines()]
print(slines)
fd.close()
for list_from_line in slines[1:]:
    last_name, first_name, middle_name, street_1, street_2, city, state, zip_code, amount, c_date, candidate_id = list_from_line[1:]
    valstoinsert = (last_name, first_name, middle_name, street_1, street_2, city, state, zip_code, int(float(amount)), datetime.fromisoformat(c_date), int(candidate_id))
    print(ins, valstoinsert)
    db.cursor().execute(ins, valstoinsert)

db.commit()

[['id', 'last_name', 'first_name', 'middle_name', 'street_1', 'street_2', 'city', 'state', 'zip', 'amount', 'date', 'candidate_id'], ['', 'Agee', 'Steven', '', '549 Laurel Branch Road', '', 'Floyd', 'VA', '24091', '500.00', '2007-06-30', '16'], ['', 'Ahrens', 'Don', '', '4034 Rennellwood Way', '', 'Pleasanton', 'CA', '94566', '250.00', '2007-05-16', '16'], ['', 'Ahrens', 'Don', '', '4034 Rennellwood Way', '', 'Pleasanton', 'CA', '94566', '50.00', '2007-06-18', '16'], ['', 'Ahrens', 'Don', '', '4034 Rennellwood Way', '', 'Pleasanton', 'CA', '94566', '100.00', '2007-06-21', '16'], ['', 'Akin', 'Charles', '', '10187 Sugar Creek Road', '', 'Bentonville', 'AR', '72712', '100.00', '2007-06-16', '16'], ['', 'Akin', 'Mike', '', '181 Baywood Lane', '', 'Monticello', 'AR', '71655', '1500.00', '2007-05-18', '16'], ['', 'Akin', 'Rebecca', '', '181 Baywood Lane', '', 'Monticello', 'AR', '71655', '500.00', '2007-05-18', '16'], ['', 'Aldridge', 'Brittni', '', '808 Capitol Square Place, SW', '', 'Wash

/var/folders/td/94qngg4x2_lcb32n6td6t7540000gn/T/ipykernel_7077/125978452.py:11: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  db.cursor().execute(ins, valstoinsert)


In [42]:
out=make_query("SELECT * FROM contributors WHERE state IN ('VA', 'MA');")
make_frame(out, legend=contributor_cols)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,526,Agee,Steven,,549 Laurel Branch Road,,Floyd,VA,24091,500,2007-06-30 00:00:00,16
1,553,Buckheit,Bruce,,8904 KAREN DR,,FAIRFAX,VA,220312731,100,2007-09-19 00:00:00,20
2,603,Ranganath,Anoop,,2507 Willard Drive,,Charlottesville,VA,22903,-100,2008-04-21 00:00:00,32
3,604,Nystrom,Michael,A,93A Fairmont Street,,Arlington,MA,02474,-503,2008-04-21 00:00:00,32
4,614,Perreault,Louise,,503 Brockridge Hunt Drive,,Hampton,VA,23666,-34,2008-04-21 00:00:00,32
5,627,Aarons,Elaine,,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,026733300,25,2008-02-26 00:00:00,34
6,628,Aarons,Elaine,,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,026733300,70,2008-02-25 00:00:00,34
7,629,Aarons,Elaine,,481 Buck Island Rd Apt 17A,APT 17A,West Yarmouth,MA,026733300,100,2008-02-08 00:00:00,34
8,671,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,,FAIRFAX,VA,220308440,50,2007-09-30 00:00:00,35
9,683,ABBOT,DAVID,M.,56 SALEM STREET,,ANDOVER,MA,018102114,200,2008-01-21 00:00:00,37


In [47]:
q = make_query("select * from contributors where last_name = 'ABREU';")
make_frame(q, contributor_cols)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,658,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,,PEARLAND,TX,775816547,50,2007-09-30 00:00:00,35
1,659,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,,PEARLAND,TX,775816547,150,2007-08-09 00:00:00,35
2,660,ABREU,KEVIN,M.,1305 GARDEN GLEN LANE,,PEARLAND,TX,775816547,50,2007-07-19 00:00:00,35


This is obviously the same dude so next time we need to clean this up so that our database does not have reprting individuals under one contributor. 